In [13]:
!pip install great_expectations ####RECEIVED EXTENSION

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 12.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 813.6/813.6 kB 12.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.1/109.1 kB 8.1 MB/s eta 0:00:00
  Attempting uninstall: ruamel.yaml
    Found existing installation: ruamel.yaml 0.17.21
    Uninstalling ruamel.yaml-0.17.21:
      Successfully uninstalled ruamel.yaml-0.17.21
  Attempting uninstall: altair
    Found existing installation: altair 5.0.1
    Uninstalling altair-5.0.1:
      Successfully uninstalled altair-5.0.1


In [280]:
## Imports
import pandas as pd
import numpy as np
import great_expectations as ge
import re

In [282]:
# Prepocessing
df = pd.read_excel("hw5_dataset.xlsx")
df.replace(-1, pd.NA, inplace=True)

def extract_salary_estimate(value):
    match = re.match(r"\$(\d+)K-\$(\d+)K \(Glassdoor est.\)", value)
    if match:
        return int(match.group(1)) * 1000, int(match.group(2)) * 1000
    return None, None


df["Min Salary Estimate"], df["Max Salary Estimate"] = zip(*df["Salary Estimate"].apply(extract_salary_estimate))
df["Min Max Diff"] = df["Max Salary Estimate"] - df["Min Salary Estimate"]

In [284]:
ge_df = ge.from_pandas(df)

In [286]:
# Step 1
print(ge_df.expect_column_values_to_be_between("Min Salary Estimate", 50000, 400000))
print(ge_df.expect_column_values_to_be_between("Max Salary Estimate", 50000, 400000))

{
  "success": false,
  "expectation_config": {
    "expectation_type": "expect_column_values_to_be_between",
    "kwargs": {
      "column": "Min Salary Estimate",
      "min_value": 50000,
      "max_value": 400000,
      "result_format": "BASIC"
    },
    "meta": {}
  },
  "result": {
    "element_count": 672,
    "missing_count": 20,
    "missing_percent": 2.976190476190476,
    "unexpected_count": 21,
    "unexpected_percent": 3.2208588957055215,
    "unexpected_percent_total": 3.125,
    "unexpected_percent_nonmissing": 3.2208588957055215,
    "partial_unexpected_list": [
      990000.0,
      31000.0,
      31000.0,
      31000.0,
      31000.0,
      31000.0,
      31000.0,
      31000.0,
      31000.0,
      31000.0,
      31000.0,
      31000.0,
      31000.0,
      31000.0,
      31000.0,
      31000.0,
      31000.0,
      31000.0,
      31000.0,
      31000.0
    ]
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
   

In [288]:
# Step 2
# If we have {A} space - {B} then all false
print(ge_df.expect_column_values_to_match_regex("Salary Estimate", r"\$(\d+)K-\$(\d+)K"))
print(ge_df.expect_column_values_to_be_between("Min Max Diff", 0, 400000 - 50000))

{
  "success": true,
  "expectation_config": {
    "expectation_type": "expect_column_values_to_match_regex",
    "kwargs": {
      "column": "Salary Estimate",
      "regex": "\\$(\\d+)K-\\$(\\d+)K",
      "result_format": "BASIC"
    },
    "meta": {}
  },
  "result": {
    "element_count": 672,
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0,
    "partial_unexpected_list": []
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}
{
  "success": false,
  "expectation_config": {
    "expectation_type": "expect_column_values_to_be_between",
    "kwargs": {
      "column": "Min Max Diff",
      "min_value": 0,
      "max_value": 350000,
      "result_format": "BASIC"
    },
    "meta": {}
  },
  "result": {
    "element_count": 672,
    "missing_count": 20,
    "mi

In [290]:
# Step 3
ge_df.expect_column_values_to_be_between("Rating", 0.0, 5.0)

{
  "success": true,
  "result": {
    "element_count": 672,
    "missing_count": 50,
    "missing_percent": 7.440476190476191,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0,
    "partial_unexpected_list": []
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

In [292]:
# Step 4
valid_states = [
    'AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'FL', 'GA', 'HI', 'ID', 'IL', 'IN',
    'IA', 'KS', 'KY', 'LA', 'ME', 'MD', 'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV',
    'NH', 'NJ', 'NM', 'NY', 'NC', 'ND', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN',
    'TX', 'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY', 'DC', "Remote", "United States",
    "Alabama", "Alaska", "Arizona", "Arkansas", "California", "Colorado", 
    "Connecticut", "Delaware", "Florida", "Georgia", "Hawaii", "Idaho", 
    "Illinois", "Indiana", "Iowa", "Kansas", "Kentucky", "Louisiana", 
    "Maine", "Maryland", "Massachusetts", "Michigan", "Minnesota", 
    "Mississippi", "Missouri", "Montana", "Nebraska", "Nevada", 
    "New Hampshire", "New Jersey", "New Mexico", "New York", 
    "North Carolina", "North Dakota", "Ohio", "Oklahoma", "Oregon", 
    "Pennsylvania", "Rhode Island", "South Carolina", "South Dakota", 
    "Tennessee", "Texas", "Utah", "Vermont", "Virginia", "Washington", 
    "West Virginia", "Wisconsin", "Wyoming"]

valid_states_regex = r"\b(?:{})\b".format("|".join(re.escape(state) for state in valid_states))
print(ge_df.expect_column_values_to_match_regex("Location", valid_states_regex))

{
  "success": false,
  "expectation_config": {
    "expectation_type": "expect_column_values_to_match_regex",
    "kwargs": {
      "column": "Location",
      "regex": "\\b(?:AL|AK|AZ|AR|CA|CO|CT|DE|FL|GA|HI|ID|IL|IN|IA|KS|KY|LA|ME|MD|MA|MI|MN|MS|MO|MT|NE|NV|NH|NJ|NM|NY|NC|ND|OH|OK|OR|PA|RI|SC|SD|TN|TX|UT|VT|VA|WA|WV|WI|WY|DC|Remote|United\\ States|Alabama|Alaska|Arizona|Arkansas|California|Colorado|Connecticut|Delaware|Florida|Georgia|Hawaii|Idaho|Illinois|Indiana|Iowa|Kansas|Kentucky|Louisiana|Maine|Maryland|Massachusetts|Michigan|Minnesota|Mississippi|Missouri|Montana|Nebraska|Nevada|New\\ Hampshire|New\\ Jersey|New\\ Mexico|New\\ York|North\\ Carolina|North\\ Dakota|Ohio|Oklahoma|Oregon|Pennsylvania|Rhode\\ Island|South\\ Carolina|South\\ Dakota|Tennessee|Texas|Utah|Vermont|Virginia|Washington|West\\ Virginia|Wisconsin|Wyoming)\\b",
      "result_format": "BASIC"
    },
    "meta": {}
  },
  "result": {
    "element_count": 672,
    "missing_count": 0,
    "missing_percent": 0.0,

In [294]:
# Step 5
# Assuming that its not literally -to- as none are in that format
size_pattern = r"^\d+ to \d+ employees$"
print(ge_df.expect_column_values_to_match_regex("Size", size_pattern))

{
  "success": false,
  "expectation_config": {
    "expectation_type": "expect_column_values_to_match_regex",
    "kwargs": {
      "column": "Size",
      "regex": "^\\d+ to \\d+ employees$",
      "result_format": "BASIC"
    },
    "meta": {}
  },
  "result": {
    "element_count": 672,
    "missing_count": 27,
    "missing_percent": 4.017857142857143,
    "unexpected_count": 98,
    "unexpected_percent": 15.193798449612403,
    "unexpected_percent_total": 14.583333333333334,
    "unexpected_percent_nonmissing": 15.193798449612403,
    "partial_unexpected_list": [
      "10000+ employees",
      "10000+ employees",
      "10000+ employees",
      "10000+ employees",
      "10000+ employees",
      "10000+ employees",
      "10000+ employees",
      "10000+ employees",
      "10000+ employees",
      "10000+ employees",
      "10000+ employees",
      "10000+ employees",
      "10000+ employees",
      "10000+ employees",
      "10000+ employees",
      "10000+ employees",
      "10

In [296]:
# Step 6
print(ge_df.expect_column_values_to_be_between("Founded", 1800, 2024))

{
  "success": false,
  "expectation_config": {
    "expectation_type": "expect_column_values_to_be_between",
    "kwargs": {
      "column": "Founded",
      "min_value": 1800,
      "max_value": 2024,
      "result_format": "BASIC"
    },
    "meta": {}
  },
  "result": {
    "element_count": 672,
    "missing_count": 118,
    "missing_percent": 17.559523809523807,
    "unexpected_count": 4,
    "unexpected_percent": 0.7220216606498195,
    "unexpected_percent_total": 0.5952380952380952,
    "unexpected_percent_nonmissing": 0.7220216606498195,
    "partial_unexpected_list": [
      1781,
      1003,
      1788,
      1781
    ]
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}


In [298]:
ge.from_pandas(df.Revenue[df.Founded == 2019]) # Indices still maintained and prof approved

,Revenue
32,Unknown / Non-Applicable
43,Unknown / Non-Applicable
77,Unknown / Non-Applicable
118,Unknown / Non-Applicable
284,Unknown / Non-Applicable
444,$25 to $50 million (USD)
488,Unknown / Non-Applicable


In [300]:
# Step 7
rev_pattern = r"Unknown / Non-Applicable"
ge.from_pandas(df.Revenue[df.Founded == 2019]).expect_column_values_to_match_regex("Revenue", rev_pattern)


{
  "success": false,
  "result": {
    "element_count": 7,
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_count": 1,
    "unexpected_percent": 14.285714285714285,
    "unexpected_percent_total": 14.285714285714285,
    "unexpected_percent_nonmissing": 14.285714285714285,
    "partial_unexpected_list": [
      "$25 to $50 million (USD)"
    ]
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

In [302]:
#indcies
elems = [444] # As we can see above and proff apporved not using GE for everything 
results = ge_df.validate()
for r in results.results:
    column = r["expectation_config"]["kwargs"]["column"]
    uni_val = r["result"]["partial_unexpected_list"]
    elems.extend(df.index[df[column].isin(uni_val)].tolist())

print(np.unique(elems))
print(len(np.unique(elems)))

[  6  31  41  45  51  60  73  80  89  90  97 108 109 138 150 155 159 164
 168 169 181 187 192 194 204 215 235 237 245 246 252 256 268 272 277 279
 286 287 292 295 299 300 321 334 336 347 354 365 370 372 383 395 401 417
 420 434 442 444 448 449 452 455 467 468 469 470 471 472 473 474 475 476
 477 478 479 480 481 482 483 484 485 486 493 508 509 535 536 548 570 578
 581 589 591 594 599 607 611 614 617 631 644 645 651 659 661]
105
